In [9]:
!pip install tomotopy

     ---------------------------------------- 5.7/5.7 MB 2.4 MB/s eta 0:00:00


In [19]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import tomotopy as tp
import matplotlib.pyplot as plt

In [2]:
# Load the Excel file with 'contract title' and 'description' columns
file_path = 'TendersWA.xlsx'
df = pd.read_excel(file_path, usecols=['Contract Title', 'Description'])
df['Combined'] = df['Contract Title'] + ' ' + df['Description']

In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yfr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yfr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')

class TextProcessor:
    def __init__(self, contraction_mapping):
        self.lemmatizer = WordNetLemmatizer()
        self.contraction_mapping = contraction_mapping
        self.stop_words = set(stopwords.words('english'))

    def preprocess_text(self, text):
        if isinstance(text, str):  
            text = self._to_lower(text)
            text = self._expand_contractions(text)
            text = self._remove_punctuation(text)
            text = self._remove_stop_words(text)
            return text
        else:
            return ''  # 返回空字符串而不是空列表

    def _to_lower(self, text):
        return text.lower()

    def _expand_contractions(self, text):
        for word, new_word in self.contraction_mapping.items():
            text = text.replace(word, new_word)
        return text

    def _remove_punctuation(self, text):
        return re.sub(r'[^\w\s]', '', text)

    def _remove_stop_words(self, text):
        return ' '.join([token for token in text.split() if token not in self.stop_words])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yfr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yfr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
processor = TextProcessor(contraction_mapping={})

df['Processed'] = df['Combined'].apply(processor.preprocess_text)

print(df[['Combined', 'Processed']])

                                                Combined  \
0      Supply and Delivery of Aspen Bedding Material ...   
1      Bar Consultancy and Staffing AGWA Rooftop Bar ...   
2      Investment Services for Art Gallery WA Through...   
3      Provision of Audience Research Provision of Au...   
4      Consultancy Services for HR Support Provision ...   
...                                                  ...   
26140  Provision of Electricity Services for WorkCove...   
26141  Redevelop WorkCover WA Conference Website in W...   
26142  Core Business Systems Technology Refresh FY201...   
26143  Core Business Systems Technology Refresh FY201...   
26144  Digital First Conciliation and Arbitration Ser...   

                                               Processed  
0      supply delivery aspen bedding material custome...  
1      bar consultancy staffing agwa rooftop bar agwa...  
2      investment services art gallery wa request art...  
3      provision audience research provisio

In [9]:
# Initialize an H-LDA model
hlda_model = tp.HLDAModel(tw=tp.TermWeight.ONE, min_df=5)

# Add preprocessed documents to the model (assuming 'Processed' column contains preprocessed text)
for index, row in df.iterrows():
    processed_text = row['Processed']
    hlda_model.add_doc(processed_text)

C:\Users\yfr\AppData\Local\Temp\ipykernel_8288\1966725409.py:7: RuntimeWarning: `words` should be an iterable of str.
  hlda_model.add_doc(processed_text)


In [ ]:
# Train the H-LDA model
for i in range(100):
    hlda_model.train(1)
    print(f"Iteration {i + 1}: Perplexity = {hlda_model.perplexity}")

In [11]:
# Print topics and their hierarchies
for level in range(hlda_model.depth):
    print(f"Level {level}:")
    for topic_id in range(hlda_model.live_k):
        if hlda_model.is_live_topic(topic_id):
            topic_words = hlda_model.get_topic_words(topic_id, level)
            parent_topic = hlda_model.parent_topic(topic_id)
            print(f"Topic {topic_id} (Parent: {parent_topic}): {topic_words}")

Level 0:
Topic 0 (Parent: -1): []
Level 1:
Topic 0 (Parent: -1): [(' ', 0.12014854699373245)]
